# Part 1 模态I腔室4数据的预处理过程[ok]
筛选后的数据可用于非时段的深度学习回归模型，如conv1d/conv2d/SimpleRNN/LSTM/BiLSTM/GRU/conv1d+SAE/LSTM+SAE/BiLSTM+SAE

处理过程包括：训练集、测试集两个部分的X字段和y值做腔室筛选、批次的时间补齐
保存结果和waferid顺序

In [1]:
# 导入包
# This makes it so that matplotlib graphics will show up within the Jupyter Notebook.
%matplotlib inline

# Standard library import

# Data Analysis Tools
import pandas as pd
import numpy as np

# Visualization Tools
import matplotlib.pyplot as plt
import seaborn as sns

# Machine Learning Tools

# File Loading Tools 
import os
import pickle

In [2]:
# 导入模态I的训练数据和测试数据，归一化
Train_data_mm = pd.read_excel("./data phm 2016/Train_data_I_mm.xlsx")
Test_data_mm = pd.read_excel("./data phm 2016/Test_data_I_mm.xlsx")
print(Train_data_mm.shape)
print(Test_data_mm.shape)

(274138, 27)
(71801, 27)


In [3]:
# 筛选CHAMBER=4腔室4的数据
Train_data_modeI_chamber4_mm = Train_data_mm.loc[Train_data_mm['CHAMBER']==4]
Test_data_modeI_chamber4_mm = Test_data_mm.loc[Test_data_mm['CHAMBER']==4]
print(Train_data_modeI_chamber4_mm.shape)
print(Test_data_modeI_chamber4_mm.shape)

(147701, 27)
(30600, 27)


### 执行保存操作要慎重

In [ ]:
# # 模态I腔室4的数据保存到本地(Tip：index=True会自动增加一列索引)
# Train_data_modeI_chamber4_mm.to_csv("./data phm 2016/Train_data_modeI_chamber4_mm.csv", index=False, header=True)
# Test_data_modeI_chamber4_mm.to_csv("./data phm 2016/Test_data_modeI_chamber4_mm.csv", index=False, header=True)

## 训练集做时间补齐

In [4]:
Train_data_modeI_chamber4_mm.columns

Index(['Unnamed: 0', 'MACHINE_ID', 'MACHINE_DATA', 'TIMESTAMP', 'WAFER_ID',
       'STAGE', 'CHAMBER', 'USAGE_OF_BACKING_FILM', 'USAGE_OF_DRESSER',
       'USAGE_OF_POLISHING_TABLE', 'USAGE_OF_DRESSER_TABLE',
       'PRESSURIZED_CHAMBER_PRESSURE', 'MAIN_OUTER_AIR_BAG_PRESSURE',
       'CENTER_AIR_BAG_PRESSURE', 'RETAINER_RING_PRESSURE',
       'RIPPLE_AIR_BAG_PRESSURE', 'USAGE_OF_MEMBRANE',
       'USAGE_OF_PRESSURIZED_SHEET', 'SLURRY_FLOW_LINE_A',
       'SLURRY_FLOW_LINE_B', 'SLURRY_FLOW_LINE_C', 'WAFER_ROTATION',
       'STAGE_ROTATION', 'HEAD_ROTATION', 'DRESSING_WATER_STATUS',
       'EDGE_AIR_BAG_PRESSURE', 'AVG_REMOVAL_RATE'],
      dtype='object')

In [5]:
Train_data_modeI_chamber4_mm.head(5)

,Unnamed: 0,MACHINE_ID,MACHINE_DATA,TIMESTAMP,WAFER_ID,STAGE,CHAMBER,USAGE_OF_BACKING_FILM,USAGE_OF_DRESSER,USAGE_OF_POLISHING_TABLE,...,USAGE_OF_PRESSURIZED_SHEET,SLURRY_FLOW_LINE_A,SLURRY_FLOW_LINE_B,SLURRY_FLOW_LINE_C,WAFER_ROTATION,STAGE_ROTATION,HEAD_ROTATION,DRESSING_WATER_STATUS,EDGE_AIR_BAG_PRESSURE,AVG_REMOVAL_RATE
0,848,2,4,481634409.7,-4113511818,A,4,0.878632,0.685894,0.704595,...,0.878632,0.053691,1.0,0.000000,0.0,0.0,0.833333,1,0.0,65.0223
1,849,2,4,481634410.3,-4113511818,A,4,0.878632,0.685894,0.704595,...,0.878632,0.053691,1.0,0.000000,0.0,0.0,0.833333,1,0.0,65.0223
2,850,2,4,481634411.3,-4113511818,A,4,0.878632,0.685894,0.704595,...,0.878632,0.053691,1.0,0.000000,0.0,0.0,0.833333,1,0.0,65.0223
3,851,2,4,481634412.3,-4113511818,A,4,0.878632,0.685894,0.704595,...,0.878632,0.053691,1.0,0.126943,0.0,0.0,0.833333,1,0.0,65.0223
4,852,2,4,481634413.7,-4113511818,A,4,0.878632,0.685894,0.704595,...,0.878632,0.053691,1.0,0.248705,0.0,0.0,0.833333,1,0.0,65.0223


In [6]:
# 数据格式做转换
# 查找最长的批次长度，做时间补齐
wafer_ids, batch_length=np.unique(Train_data_modeI_chamber4_mm['WAFER_ID'], return_counts=True)
max_batch_length = max(batch_length)
print('批次最大长度：', max_batch_length)
print(batch_length)

批次最大长度： 263
[187 187 188 187 189 190 193 193 192 186 189 181 189 197 186 192 197 189
 183 184 193 196 172 182 184 211 180 197 194 197 193 178 186 193 194 185
 194 202 198 191 183 197 195 192 198 184 193 190 178 186 191 186 187 191
 190 190 188 190 208 198 198 191 198 205 196 193 194 192 186 184 190 187
 191 189 185 187 184 183 185 174 182 180 181 177 180 184 179 182 181 178
 169 184 184 181 188 175 185 187 210 175 179 192 195 190 188 191 177 193
 186 190 191 193 196 196 191 188 187 178 187 177 188 189 188 197 187 194
 188 189 179 182 187 184 186 186 184 186 181 184 192 188 197 189 191 195
 187 194 191 189 190 198 200 188 191 197 188 186 194 188 195 187 186 186
 194 192 173 180 187 192 186 191 190 189 187 191 182 188 180 195 198 188
 193 191 188 187 188 174 187 194 189 189 179 169 177 184 191 182 184 185
 182 186 186 193 190 179 191 193 194 177 186 187 177 193 194 203 190 191
 200 192 194 195 190 189 194 199 193 195 193 190 191 193 201 199 200 195
 191 184 187 193 195 198 183 182 181 18

In [7]:
# drop columns
X_train = Train_data_modeI_chamber4_mm.drop(columns=['Unnamed: 0', 'MACHINE_ID', 'MACHINE_DATA', 'TIMESTAMP', 'STAGE', 'CHAMBER'])
X_train.head()

,WAFER_ID,USAGE_OF_BACKING_FILM,USAGE_OF_DRESSER,USAGE_OF_POLISHING_TABLE,USAGE_OF_DRESSER_TABLE,PRESSURIZED_CHAMBER_PRESSURE,MAIN_OUTER_AIR_BAG_PRESSURE,CENTER_AIR_BAG_PRESSURE,RETAINER_RING_PRESSURE,RIPPLE_AIR_BAG_PRESSURE,...,USAGE_OF_PRESSURIZED_SHEET,SLURRY_FLOW_LINE_A,SLURRY_FLOW_LINE_B,SLURRY_FLOW_LINE_C,WAFER_ROTATION,STAGE_ROTATION,HEAD_ROTATION,DRESSING_WATER_STATUS,EDGE_AIR_BAG_PRESSURE,AVG_REMOVAL_RATE
0,-4113511818,0.878632,0.685894,0.704595,0.0,0.0,0.0,0.0,0.0,0.0,...,0.878632,0.053691,1.0,0.000000,0.0,0.0,0.833333,1,0.0,65.0223
1,-4113511818,0.878632,0.685894,0.704595,0.0,0.0,0.0,0.0,0.0,0.0,...,0.878632,0.053691,1.0,0.000000,0.0,0.0,0.833333,1,0.0,65.0223
2,-4113511818,0.878632,0.685894,0.704595,0.0,0.0,0.0,0.0,0.0,0.0,...,0.878632,0.053691,1.0,0.000000,0.0,0.0,0.833333,1,0.0,65.0223
3,-4113511818,0.878632,0.685894,0.704595,0.0,0.0,0.0,0.0,0.0,0.0,...,0.878632,0.053691,1.0,0.126943,0.0,0.0,0.833333,1,0.0,65.0223
4,-4113511818,0.878632,0.685894,0.704595,0.0,0.0,0.0,0.0,0.0,0.0,...,0.878632,0.053691,1.0,0.248705,0.0,0.0,0.833333,1,0.0,65.0223


In [8]:
X_train.set_index('WAFER_ID')
X_train.head()

,WAFER_ID,USAGE_OF_BACKING_FILM,USAGE_OF_DRESSER,USAGE_OF_POLISHING_TABLE,USAGE_OF_DRESSER_TABLE,PRESSURIZED_CHAMBER_PRESSURE,MAIN_OUTER_AIR_BAG_PRESSURE,CENTER_AIR_BAG_PRESSURE,RETAINER_RING_PRESSURE,RIPPLE_AIR_BAG_PRESSURE,...,USAGE_OF_PRESSURIZED_SHEET,SLURRY_FLOW_LINE_A,SLURRY_FLOW_LINE_B,SLURRY_FLOW_LINE_C,WAFER_ROTATION,STAGE_ROTATION,HEAD_ROTATION,DRESSING_WATER_STATUS,EDGE_AIR_BAG_PRESSURE,AVG_REMOVAL_RATE
0,-4113511818,0.878632,0.685894,0.704595,0.0,0.0,0.0,0.0,0.0,0.0,...,0.878632,0.053691,1.0,0.000000,0.0,0.0,0.833333,1,0.0,65.0223
1,-4113511818,0.878632,0.685894,0.704595,0.0,0.0,0.0,0.0,0.0,0.0,...,0.878632,0.053691,1.0,0.000000,0.0,0.0,0.833333,1,0.0,65.0223
2,-4113511818,0.878632,0.685894,0.704595,0.0,0.0,0.0,0.0,0.0,0.0,...,0.878632,0.053691,1.0,0.000000,0.0,0.0,0.833333,1,0.0,65.0223
3,-4113511818,0.878632,0.685894,0.704595,0.0,0.0,0.0,0.0,0.0,0.0,...,0.878632,0.053691,1.0,0.126943,0.0,0.0,0.833333,1,0.0,65.0223
4,-4113511818,0.878632,0.685894,0.704595,0.0,0.0,0.0,0.0,0.0,0.0,...,0.878632,0.053691,1.0,0.248705,0.0,0.0,0.833333,1,0.0,65.0223


In [9]:
wafer_number = len(wafer_ids)
columns_var = set(X_train.columns)-set(['WAFER_ID', 'AVG_REMOVAL_RATE'])
variable_number = len(columns_var)
print('晶圆个数：', wafer_number)
print('字段个数：', variable_number)

晶圆个数： 798
字段个数： 19


In [10]:
# reshape train data
# X_train_r第1维是批次个数，第2维是每个批次的采样时间，不足最长采样时间的用0补齐，第3维是字段的个数
X_train_r = np.zeros((wafer_number, max_batch_length, variable_number))

In [11]:
y_train = np.zeros((wafer_number, 1))
y_train[0]

array([0.])

In [15]:
wafer_id_train = []

In [16]:
i = 0
for name, group in X_train.groupby(['WAFER_ID']):
    wafer_id_train.append(name)
    one_wafer = group.drop(columns=['WAFER_ID', 'AVG_REMOVAL_RATE'])
    y_train[i] = group['AVG_REMOVAL_RATE'].iloc[0]
    print(y_train[i])
    one_wafer_length = one_wafer.shape[0]
#     if one_wafer_length < max_batch_length:
    df_tmp = pd.DataFrame(np.zeros((max_batch_length-one_wafer_length, len(columns_var))), columns=columns_var)
    X_train_r[i, :, :] = pd.concat([one_wafer, df_tmp])        
    i+=1

[68.8818]
[75.34995]
[78.33015]
[79.8045]
[67.42425]
[64.521]
[72.37185]
[70.0665]
[66.02895]
[74.21805]
[72.79755]
[61.94655]
[78.33675]
[60.6567]
[62.05515]
[61.1604]
[58.3518]
[63.24495]
[57.9618]
[61.22265]
[57.86655]
[57.8484]
[64.35705]
[60.3432]
[61.0698]
[62.62545]
[61.6548]
[65.16345]
[53.6943]
[53.7738]
[53.42655]
[68.09415]
[79.85925]
[58.9389]
[65.65575]
[76.0956]
[64.4562]
[55.0086]
[56.9127]
[64.90665]
[61.4217]
[63.6816]
[58.9947]
[59.3889]
[56.9673]
[78.65925]
[58.81275]
[75.0489]
[82.58775]
[78.4977]
[74.30415]
[70.3578]
[69.5982]
[72.10635]
[71.9658]
[70.1346]
[69.7959]
[66.46995]
[62.3022]
[65.0223]
[58.27905]
[53.64735]
[59.1954]
[59.8572]
[61.04565]
[55.60095]
[56.95725]
[56.09865]
[76.88835]
[76.6485]
[79.4853]
[64.21455]
[54.59835]
[73.4904]
[76.0023]
[80.33985]
[69.92775]
[75.37605]
[74.95455]
[79.15335]
[86.16075]
[83.77365]
[84.1707]
[84.68055]
[83.1678]
[85.86135]
[82.98195]
[85.27395]
[88.70205]
[74.61195]
[78.7752]
[78.9513]
[79.17435]
[73.61025]
[76.53705]

In [17]:
X_train_r.shape

(798, 263, 19)

In [18]:
print(y_train.shape)

(798, 1)


In [19]:
y_train

array([[68.8818 ],
       [75.34995],
       [78.33015],
       [79.8045 ],
       [67.42425],
       [64.521  ],
       [72.37185],
       [70.0665 ],
       [66.02895],
       [74.21805],
       [72.79755],
       [61.94655],
       [78.33675],
       [60.6567 ],
       [62.05515],
       [61.1604 ],
       [58.3518 ],
       [63.24495],
       [57.9618 ],
       [61.22265],
       [57.86655],
       [57.8484 ],
       [64.35705],
       [60.3432 ],
       [61.0698 ],
       [62.62545],
       [61.6548 ],
       [65.16345],
       [53.6943 ],
       [53.7738 ],
       [53.42655],
       [68.09415],
       [79.85925],
       [58.9389 ],
       [65.65575],
       [76.0956 ],
       [64.4562 ],
       [55.0086 ],
       [56.9127 ],
       [64.90665],
       [61.4217 ],
       [63.6816 ],
       [58.9947 ],
       [59.3889 ],
       [56.9673 ],
       [78.65925],
       [58.81275],
       [75.0489 ],
       [82.58775],
       [78.4977 ],
       [74.30415],
       [70.3578 ],
       [69.5

In [21]:
wafer_id_train # 结果是按照waferid从小到大排序

[-4230160598,
 -4230160428,
 -4230160424,
 -4230160408,
 -4228160702,
 -4228160690,
 -4228160686,
 -4228160682,
 -4228160678,
 -4228160670,
 -4228160662,
 -4228160658,
 -4228160642,
 -4228160604,
 -4228160600,
 -4228160596,
 -4228160580,
 -4226160424,
 -4226160416,
 -4226160396,
 -4224160694,
 -4224160690,
 -4224160662,
 -4224160654,
 -4224160650,
 -4224160646,
 -4224160600,
 -4222160448,
 -4222160428,
 -4222160420,
 -4222160392,
 -4220160702,
 -4220160694,
 -4220160674,
 -4220160650,
 -4220160646,
 -4220160604,
 -4216160702,
 -4216160694,
 -4216160670,
 -4216160666,
 -4216160662,
 -4216160658,
 -4216160650,
 -4216160642,
 -4216160640,
 -4216160624,
 -4210160444,
 -4208160362,
 -4141512120,
 -4141512096,
 -4141512072,
 -4139512004,
 -4139511866,
 -4139511854,
 -4139511850,
 -4139511842,
 -4139511826,
 -4113511826,
 -4113511818,
 -4113511814,
 -4113511808,
 -4113511796,
 -4113511792,
 -4113511780,
 -4113511756,
 -4113511752,
 -4113511748,
 -4111512062,
 -4111512058,
 -4111512050,
 -4111

### <font color='red'> 慎重执行保存 </font>

In [ ]:
# # 保存数组到本地
# np.save("./data phm 2016/X_train_r_modeI_chamber4_mm.npy", X_train_r)
# np.save("./data phm 2016/y_train_modeI_chamber4_mm.npy", y_train)

In [35]:
# with open("./data phm 2016/wafer_id_train_modeI_chamber4_mm.pkl", 'wb') as f:
#     pickle.dump(wafer_id_train, f)

## 测试集做时间补齐

In [22]:
# 测试数据格式做转换
# 查找最长的批次长度202<训练集的最长批次长度263，仍然使用263批次长度做时间补齐
wafer_ids_test, batch_length_test=np.unique(Test_data_modeI_chamber4_mm['WAFER_ID'], return_counts=True)
max_batch_length_test = max(batch_length_test)
print('批次最大长度：', max_batch_length_test)
print(batch_length_test)

批次最大长度： 202
[198 193 188 188 191 188 202 190 184 169 181 193 190 177 187 198 194 191
 182 187 189 192 186 188 186 194 187 191 192 191 188 188 194 196 189 194
 187 185 187 183 192 196 200 180 193 182 193 194 196 179 191 201 194 183
 181 185 187 180 179 181 184 190 186 186 183 185 181 180 187 182 181 182
 183 179 168 180 184 179 172 186 186 169 181 182 183 181 167 180 169 166
 177 181 186 183 176 181 186 192 184 182 182 179 182 180 189 185 182 183
 190 190 189 178 190 187 176 178 183 176 184 183 182 182 185 185 192 183
 183 181 187 181 185 193 184 197 200 174 168 186 192 190 187 186 187 188
 193 183 189 190 187 178 188 187 189 184 188 191 173 181 201 181 175 188
 196 190 189]


In [23]:
# drop columns
X_test = Test_data_modeI_chamber4_mm.drop(columns=['Unnamed: 0', 'MACHINE_ID', 'MACHINE_DATA', 'TIMESTAMP', 'STAGE', 'CHAMBER'])
X_test.head()

,WAFER_ID,USAGE_OF_BACKING_FILM,USAGE_OF_DRESSER,USAGE_OF_POLISHING_TABLE,USAGE_OF_DRESSER_TABLE,PRESSURIZED_CHAMBER_PRESSURE,MAIN_OUTER_AIR_BAG_PRESSURE,CENTER_AIR_BAG_PRESSURE,RETAINER_RING_PRESSURE,RIPPLE_AIR_BAG_PRESSURE,...,USAGE_OF_PRESSURIZED_SHEET,SLURRY_FLOW_LINE_A,SLURRY_FLOW_LINE_B,SLURRY_FLOW_LINE_C,WAFER_ROTATION,STAGE_ROTATION,HEAD_ROTATION,DRESSING_WATER_STATUS,EDGE_AIR_BAG_PRESSURE,AVG_REMOVAL_RATE
0,-4224160592,0.885617,0.690257,0.894967,0.001372,0.0,0.0,0.0,0.0,0.0,...,0.885617,0.053691,1,0.000000,0.0,0.0,0.833333,1,0.0,56.1786
1,-4224160592,0.885617,0.690257,0.894967,0.001372,0.0,0.0,0.0,0.0,0.0,...,0.885617,0.083893,1,0.049223,0.0,0.0,0.833333,1,0.0,56.1786
2,-4224160592,0.885617,0.690257,0.894967,0.001372,0.0,0.0,0.0,0.0,0.0,...,0.885617,0.335570,1,0.222798,0.0,0.0,0.833333,1,0.0,56.1786
3,-4224160592,0.885617,0.690257,0.894967,0.001372,0.0,0.0,0.0,0.0,0.0,...,0.885617,0.412752,1,0.279793,0.0,0.0,0.833333,1,0.0,56.1786
4,-4224160592,0.885617,0.690257,0.894967,0.001372,0.0,0.0,0.0,0.0,0.0,...,0.885617,0.469799,1,0.339378,0.0,0.0,0.833333,1,0.0,56.1786


In [24]:
X_test.set_index('WAFER_ID')
X_test.head()

,WAFER_ID,USAGE_OF_BACKING_FILM,USAGE_OF_DRESSER,USAGE_OF_POLISHING_TABLE,USAGE_OF_DRESSER_TABLE,PRESSURIZED_CHAMBER_PRESSURE,MAIN_OUTER_AIR_BAG_PRESSURE,CENTER_AIR_BAG_PRESSURE,RETAINER_RING_PRESSURE,RIPPLE_AIR_BAG_PRESSURE,...,USAGE_OF_PRESSURIZED_SHEET,SLURRY_FLOW_LINE_A,SLURRY_FLOW_LINE_B,SLURRY_FLOW_LINE_C,WAFER_ROTATION,STAGE_ROTATION,HEAD_ROTATION,DRESSING_WATER_STATUS,EDGE_AIR_BAG_PRESSURE,AVG_REMOVAL_RATE
0,-4224160592,0.885617,0.690257,0.894967,0.001372,0.0,0.0,0.0,0.0,0.0,...,0.885617,0.053691,1,0.000000,0.0,0.0,0.833333,1,0.0,56.1786
1,-4224160592,0.885617,0.690257,0.894967,0.001372,0.0,0.0,0.0,0.0,0.0,...,0.885617,0.083893,1,0.049223,0.0,0.0,0.833333,1,0.0,56.1786
2,-4224160592,0.885617,0.690257,0.894967,0.001372,0.0,0.0,0.0,0.0,0.0,...,0.885617,0.335570,1,0.222798,0.0,0.0,0.833333,1,0.0,56.1786
3,-4224160592,0.885617,0.690257,0.894967,0.001372,0.0,0.0,0.0,0.0,0.0,...,0.885617,0.412752,1,0.279793,0.0,0.0,0.833333,1,0.0,56.1786
4,-4224160592,0.885617,0.690257,0.894967,0.001372,0.0,0.0,0.0,0.0,0.0,...,0.885617,0.469799,1,0.339378,0.0,0.0,0.833333,1,0.0,56.1786


In [25]:
wafer_number_test = len(wafer_ids_test)
print('测试集晶圆个数：', wafer_number_test)

测试集晶圆个数： 165


In [26]:
# reshape test data
# X_test_r第1维是批次个数，第2维是每个批次的采样时间，不足最长采样时间的用0补齐，第3维是字段的个数
X_test_r = np.zeros((wafer_number_test, max_batch_length, variable_number))
X_test_r.shape

(165, 263, 19)

In [27]:
y_test = np.zeros((wafer_number_test, 1))
y_test[0]

array([0.])

In [28]:
wafer_id_test = []

In [29]:
i = 0
for name, group in X_test.groupby(['WAFER_ID']):
    wafer_id_test.append(name)
    one_wafer = group.drop(columns=['WAFER_ID', 'AVG_REMOVAL_RATE'])
    y_test[i] = group['AVG_REMOVAL_RATE'].iloc[0]
    print(y_test[i])
    one_wafer_length = one_wafer.shape[0]
#     if one_wafer_length < max_batch_length:
    df_tmp = pd.DataFrame(np.zeros((max_batch_length-one_wafer_length, len(columns_var))), columns=columns_var)
    X_test_r[i, :, :] = pd.concat([one_wafer, df_tmp])        
    i+=1

[60.44715]
[57.2523]
[56.1786]
[69.84885]
[70.56045]
[70.04175]
[61.41315]
[82.56705]
[75.29955]
[62.7504]
[78.48405]
[70.2735]
[69.9954]
[65.64585]
[70.7943]
[69.63975]
[66.5856]
[70.79865]
[72.97155]
[69.4038]
[70.5174]
[73.0947]
[68.36535]
[74.7423]
[72.5712]
[66.47775]
[68.01585]
[63.6735]
[67.9998]
[72.10245]
[72.18225]
[70.51065]
[71.8215]
[72.47895]
[70.7502]
[71.17155]
[74.5998]
[78.03345]
[76.62405]
[78.6405]
[63.4305]
[64.84635]
[58.3209]
[59.4636]
[63.44025]
[69.8505]
[66.8805]
[61.5069]
[65.17935]
[63.26625]
[54.5025]
[67.7304]
[56.64495]
[73.84995]
[74.92635]
[81.38565]
[69.91755]
[75.85785]
[70.7889]
[72.7815]
[75.13665]
[77.3532]
[76.8978]
[77.4405]
[72.5799]
[76.8822]
[77.7087]
[74.895]
[74.42475]
[71.8617]
[69.66795]
[66.9333]
[67.95825]
[74.1255]
[74.94945]
[70.8963]
[71.74995]
[81.08355]
[78.46215]
[79.3176]
[77.1681]
[83.6712]
[80.5788]
[73.2867]
[81.1596]
[72.6753]
[79.563]
[74.47875]
[78.43335]
[74.5197]
[78.91875]
[79.61205]
[81.3951]
[73.16355]
[78.23445]
[86.71

In [30]:
print(X_test_r.shape)
print(y_test.shape)
print(y_test)

(165, 263, 19)
(165, 1)
[[60.44715]
 [57.2523 ]
 [56.1786 ]
 [69.84885]
 [70.56045]
 [70.04175]
 [61.41315]
 [82.56705]
 [75.29955]
 [62.7504 ]
 [78.48405]
 [70.2735 ]
 [69.9954 ]
 [65.64585]
 [70.7943 ]
 [69.63975]
 [66.5856 ]
 [70.79865]
 [72.97155]
 [69.4038 ]
 [70.5174 ]
 [73.0947 ]
 [68.36535]
 [74.7423 ]
 [72.5712 ]
 [66.47775]
 [68.01585]
 [63.6735 ]
 [67.9998 ]
 [72.10245]
 [72.18225]
 [70.51065]
 [71.8215 ]
 [72.47895]
 [70.7502 ]
 [71.17155]
 [74.5998 ]
 [78.03345]
 [76.62405]
 [78.6405 ]
 [63.4305 ]
 [64.84635]
 [58.3209 ]
 [59.4636 ]
 [63.44025]
 [69.8505 ]
 [66.8805 ]
 [61.5069 ]
 [65.17935]
 [63.26625]
 [54.5025 ]
 [67.7304 ]
 [56.64495]
 [73.84995]
 [74.92635]
 [81.38565]
 [69.91755]
 [75.85785]
 [70.7889 ]
 [72.7815 ]
 [75.13665]
 [77.3532 ]
 [76.8978 ]
 [77.4405 ]
 [72.5799 ]
 [76.8822 ]
 [77.7087 ]
 [74.895  ]
 [74.42475]
 [71.8617 ]
 [69.66795]
 [66.9333 ]
 [67.95825]
 [74.1255 ]
 [74.94945]
 [70.8963 ]
 [71.74995]
 [81.08355]
 [78.46215]
 [79.3176 ]
 [77.1681 ]
 [83

In [32]:
print(wafer_id_test)

[-4226160404, -4224160686, -4224160592, -4220160654, -4141512092, -4139511862, -4113511760, -4111512042, -4033511554, -4033511552, -4033511540, -4027511798, -4025511544, -4023511616, -4023511604, -4021511610, -4019511762, -4019511616, -3780722714, -2701400544, -2699400360, -2579249818, -2569249796, -2567249784, -2565249986, -2149929676, -2149929668, -2143929736, -2139929776, -2131929490, -2131929472, -2131929444, -2129929678, -2039929208, -2033929454, -2033929446, -2033929442, -1714100714, -903170202, -903170120, -899170194, -899170172, -899170164, -879170266, -879170084, -879170080, -879170064, -877170362, -877170314, -873170358, -873170342, -873170310, -873170244, 29494112, 29494128, 31494172, 33494084, 33494140, 33494174, 37494374, 37494400, 37494424, 37494440, 41494382, 41494448, 43494154, 43494198, 43494396, 1270253696, 1286253650, 1286253740, 1288253466, 1288253644, 1292253510, 1294253780, 1294253804, 1300253506, 1304253506, 1304253514, 1304253538, 1306253808, 1306253820, 1308253

### <font color='red'> 慎重执行保存代码 </font>

In [ ]:
# # 保存测试数据到本地
# np.save("./data phm 2016/X_test_r_modeI_chamber4_mm.npy", X_test_r)
# np.save("./data phm 2016/y_test_modeI_chamber4_mm.npy", y_test)

In [34]:
# with open("./data phm 2016/wafer_id_test_modeI_chamber4_mm.pkl", 'wb') as f:
#     pickle.dump(wafer_id_test, f)

# Part 2 三个模态的数据全部未展开，用于深度学习模型训练[ok]
模态I(CHAMBER=4/5/6,STAGE=A)

模态II(CHAMBER=4/5/6,STAGE=B)

模态III(CHAMBER=1/2/3,STAGE=A,删除'AVG_REMOVAL_RATE'<1000的outlier)

可用于的深度学习模型有：conv1d/conv2d/rnn/lstm/bilstm/gru

In [107]:
df_three_mode_train = pd.read_csv('./data phm 2016/df_three_mode_train.csv')
df_three_mode_test = pd.read_csv('./data phm 2016/df_three_mode_test.csv')

##### 训练集check批次最长的采样个数
注意：这里有这样一个问题，同一个WAFER_ID可能同时存在于不同的mode中，比如modeI和modeII中都有同一个WAFER_ID，所以index需要WAFER_ID和Mode两个一块儿标识

In [108]:
df_train_count = df_three_mode_train.groupby(['WAFER_ID', 'Mode']).count()['MACHINE_ID']
batch_length = df_train_count.values
wafer_ids_mode = df_train_count.index
max_batch_length = max(batch_length)
print('训练集wafer-mode数量: ', len(batch_length))
print('批次最大长度: ', max_batch_length)
print(batch_length)

训练集wafer-mode数量:  1977
批次最大长度:  5492
[346 256 359 ... 349 365 360]


In [109]:
strange_wafer_mode_list = wafer_ids_mode[batch_length>500] # 认为长度超过2000的是outlier
print('strange wafer mode', strange_wafer_mode_list)

strange wafer mode MultiIndex([(-4228160596,  'II'),
            (-4226160408,  'II'),
            (-4019511766,  'II'),
            (   31494350,   'I'),
            (  665222804, 'III'),
            ( 1475739292,  'II'),
            ( 1836206950, 'III'),
            ( 2056207562, 'III'),
            ( 2056207566, 'III'),
            ( 3015014228,  'II'),
            ( 3021014294,  'II'),
            ( 4179773518,   'I'),
            ( 4181773366,   'I'),
            ( 4201773386,  'II')],
           names=['WAFER_ID', 'Mode'])


In [110]:
# plt.figure()
# plt.plot(range(0, 335), df_three_mode_train[(df_three_mode_train['WAFER_ID']==3021014508) & (df_three_mode_train['Mode']=='I')]['RETAINER_RING_PRESSURE'])
# plt.show()   

In [111]:
# plt.figure()
# plt.plot(range(0, 5205), df_three_mode_train[(df_three_mode_train['WAFER_ID']==3015014228) & (df_three_mode_train['Mode']=='II')]['RETAINER_RING_PRESSURE'])
# plt.show()     

In [112]:
# 删除strange wafer list
df_train = df_three_mode_train.copy()
for wafer_id, mode in strange_wafer_mode_list:
    print(wafer_id, mode)
    df_train = df_train[~((df_train['WAFER_ID']==wafer_id) & (df_train['Mode']==mode))]

-4228160596 II
-4226160408 II
-4019511766 II
31494350 I
665222804 III
1475739292 II
1836206950 III
2056207562 III
2056207566 III
3015014228 II
3021014294 II
4179773518 I
4181773366 I
4201773386 II


In [113]:
# 重新查找最长的批次长度
df_train_count = df_train.groupby(['WAFER_ID', 'Mode']).count()['MACHINE_ID']
batch_length = df_train_count.values
wafer_ids_mode = df_train_count.index
max_batch_length = max(batch_length)
print('训练集wafer-mode数量: ', len(batch_length))
print('批次最大长度: ', max_batch_length)
print(batch_length)

训练集wafer-mode数量:  1963
批次最大长度:  462
[346 256 359 ... 349 365 360]


##### 测试集check批次最长的采样个数

In [115]:
# 同样的方式，查找测试集最长的批次长度
df_test_count = df_three_mode_test.groupby(['WAFER_ID', 'Mode']).count()['MACHINE_ID']
batch_length_test = df_test_count.values
wafer_ids_mode_test = df_test_count.index
max_batch_length_test = max(batch_length_test)
print('测试集wafer-mode数量: ', len(batch_length_test))
print('批次最大长度: ', max_batch_length_test)
print(batch_length_test)

测试集wafer-mode数量:  424
批次最大长度:  14678
[  357   350   356   346   373   354   353   346   289   344   346   362
   347   343  1321   348   350   344   352   342   342   344   343   344
   341   322   339   349   349   353   361   348   335   345   355   350
   359   347   353   360   353   353   350   366   348   339   337   359
   344   344   343   349   344   344   354   339   347   342   347   363
   363   352   354   345   346   347   350   343   351   339   341   350
   348   354   346   349   382   342   343   344   348   287   281   286
   345   339   348   362   348   352   358   334   343   336   339   360
   354   353   340   349   353   354   353   339   348   362   379   358
   353   345   337   339   337   329   345   339   347   337   334   338
   343   345   341   342   340   339   339   326   340   337   339   349
   280   282   284   296   282   280   277   287   279   278   276   287
   276   272   277   283   278   271   283   270   281   277   290   277
   344   352  

In [116]:
strange_wafer_mode_list_test = wafer_ids_mode_test[batch_length_test>500] # 认为长度超过2000的是outlier
print('strange wafer mode', strange_wafer_mode_list_test)

strange wafer mode MultiIndex([(-4113511760, 'I'),
            ( 2925014266, 'I')],
           names=['WAFER_ID', 'Mode'])


In [117]:
# 删除strange wafer list
df_test = df_three_mode_test.copy()
for wafer_id, mode in strange_wafer_mode_list_test:
    print(wafer_id, mode)
    df_test = df_test[~((df_test['WAFER_ID']==wafer_id) & (df_test['Mode']==mode))]

-4113511760 I
2925014266 I


In [118]:
# 重新查找最长的批次长度
df_test_count = df_test.groupby(['WAFER_ID', 'Mode']).count()['MACHINE_ID']
batch_length_test = df_test_count.values
wafer_ids_mode_test = df_test_count.index
max_batch_length_test = max(batch_length_test)
print('训练集wafer-mode数量: ', len(batch_length_test))
print('批次最大长度: ', max_batch_length_test)
print(batch_length_test)

训练集wafer-mode数量:  422
批次最大长度:  382
[357 350 356 346 373 354 353 346 289 344 346 362 347 343 348 350 344 352
 342 342 344 343 344 341 322 339 349 349 353 361 348 335 345 355 350 359
 347 353 360 353 353 350 366 348 339 337 359 344 344 343 349 344 344 354
 339 347 342 347 363 363 352 354 345 346 347 350 343 351 339 341 350 348
 354 346 349 382 342 343 344 348 287 281 286 345 339 348 362 348 352 358
 334 343 336 339 360 354 353 340 349 353 354 353 339 348 362 379 358 353
 345 337 339 337 329 345 339 347 337 334 338 343 345 341 342 340 339 339
 326 340 337 339 349 280 282 284 296 282 280 277 287 279 278 276 287 276
 272 277 283 278 271 283 270 281 277 290 277 344 352 337 340 338 338 343
 337 342 338 349 346 339 347 340 335 324 336 335 345 335 329 335 344 341
 323 337 340 340 337 324 337 324 321 346 332 341 336 347 336 340 335 337
 337 332 339 346 344 339 335 352 345 342 344 352 349 346 342 317 346 345
 340 346 339 336 340 345 337 336 337 340 359 336 329 339 277 274 266 265
 275 271 276 278

##### reshape训练集和测试集的批次采样长度

In [132]:
# # 训练集批次最长采样个数是462, 测试集批次最长采样个数是382, 所以选择长的462

In [120]:
cols_all = list(df_three_mode_train.columns)
cols_unit = ['WAFER_ID', 'Mode']
col_y = ['AVG_REMOVAL_RATE']
cols_status = ['Unnamed: 0', 'MACHINE_ID', 'MACHINE_DATA', 'TIMESTAMP', 'STAGE', 'CHAMBER']
cols_feature = list(set(cols_all) - set(cols_unit) - set(col_y) - set(cols_status))

In [135]:
# 训练集: 筛选下去outlier的剩余的数据
X_train = df_train.loc[:, np.hstack([cols_unit, cols_feature, col_y])].copy()

wafer_number = len(X_train.groupby(['WAFER_ID', 'Mode']))
max_batch_length = 462
variable_number = len(cols_feature)
print('训练集样本个数: ', wafer_number)
print('批次最长长度: ', max_batch_length)
print('变量个数: ', variable_number)

# 将训练集reshape
# reshape train data
# X_train_r第1维是批次个数，第2维是每个批次的采样时间，不足最长采样时间的用0补齐，第3维是字段的个数
X_train_r = np.zeros((wafer_number, max_batch_length, variable_number))

y_train = np.zeros((wafer_number, 1))

wafer_id_mode_train = []

i = 0
for name, group in X_train.groupby(['WAFER_ID', 'Mode']):
    wafer_id_mode_train.append(name)
    one_wafer = group.drop(columns=['WAFER_ID', 'AVG_REMOVAL_RATE', 'Mode'])
    y_train[i] = group['AVG_REMOVAL_RATE'].iloc[0]
    print(y_train[i])
    one_wafer_length = one_wafer.shape[0]
#     if one_wafer_length < max_batch_length:
    df_tmp = pd.DataFrame(np.zeros((max_batch_length-one_wafer_length, len(cols_feature))), columns=cols_feature)
    X_train_r[i, :, :] = pd.concat([one_wafer, df_tmp])        
    i+=1

X_train_r.shape

训练集样本个数:  1963
批次最长长度:  462
变量个数:  19
[68.8818]
[70.0533]
[54.3072]
[75.34995]
[78.33015]
[75.97095]
[81.83385]
[79.8045]
[83.0358]
[67.42425]
[69.62205]
[74.91585]
[69.89805]
[64.521]
[68.48055]
[72.37185]
[70.2468]
[70.0665]
[66.02895]
[74.21805]
[69.83175]
[72.79755]
[61.59885]
[61.94655]
[72.771]
[78.33675]
[68.7276]
[72.48105]
[60.6567]
[62.05515]
[72.4548]
[61.1604]
[58.3518]
[73.3764]
[63.24495]
[64.1814]
[57.9618]
[72.28245]
[71.91825]
[61.22265]
[77.4597]
[65.30385]
[57.86655]
[65.64675]
[57.8484]
[66.4284]
[64.35705]
[69.36975]
[60.3432]
[61.0698]
[65.64225]
[62.62545]
[61.6548]
[61.18935]
[60.39825]
[75.86415]
[71.907]
[59.78145]
[72.45435]
[57.58005]
[85.6827]
[65.16345]
[78.6462]
[53.6943]
[79.52355]
[53.7738]
[53.42655]
[68.09415]
[84.64815]
[79.85925]
[82.76775]
[69.39165]
[74.18535]
[58.9389]
[65.65575]
[76.0956]
[73.74045]
[64.4562]
[82.51935]
[55.0086]
[56.9127]
[60.7173]
[64.90665]
[67.6425]
[61.4217]
[63.6816]
[58.9947]
[59.3889]
[64.34745]
[56.9673]
[78.65925]
[81.

[80.00865]
[78.1785]
[90.0669]
[89.44185]
[83.41635]
[84.3666]
[82.58805]
[81.67665]
[87.9402]
[88.58115]
[82.95795]
[81.65685]
[85.8987]
[85.3224]
[81.51435]
[88.63005]
[78.7743]
[71.1882]
[91.0695]
[85.6653]
[80.0772]
[91.19205]
[96.1875]
[78.72615]
[82.82955]
[91.11525]
[93.7632]
[87.04485]
[85.60095]
[81.8019]
[85.10175]
[82.33515]
[71.7375]
[80.1117]
[87.6795]
[90.63855]
[81.0465]
[92.46015]
[78.78465]
[84.2172]
[89.1201]
[90.8298]
[93.6186]
[76.2177]
[87.45645]
[83.76855]
[93.4476]
[77.8254]
[73.29225]
[78.66705]
[94.0245]
[82.707]
[71.8506]
[91.5012]
[86.0289]
[81.7002]
[81.3804]
[84.4605]
[81.9657]
[75.52995]
[85.91745]
[79.01535]
[83.09295]
[82.02915]
[79.8369]
[77.3766]
[86.72175]
[85.81695]
[87.6009]
[85.03935]
[89.004]
[82.425]
[79.69305]
[64.04925]
[65.13795]
[84.86895]
[82.54545]
[77.03835]
[77.08095]
[75.5517]
[87.56835]
[96.5514]
[73.81275]
[82.2507]
[80.9979]
[91.42335]
[83.71605]
[70.49925]
[83.07855]
[83.4507]
[83.6733]
[77.2362]
[87.9891]
[79.0431]
[70.29945]
[88.53

[75.3621]
[83.37885]
[81.73245]
[81.92955]
[82.4331]
[76.59255]
[73.3311]
[85.1316]
[78.1875]
[85.93425]
[86.6982]
[84.9375]
[84.4305]
[84.0453]
[71.9487]
[83.4648]
[89.40495]
[73.0263]
[71.29815]
[84.6372]
[86.10045]
[77.16615]
[82.80465]
[78.8424]
[86.90985]
[82.8237]
[76.3008]
[79.15485]
[90.38175]
[75.2973]
[76.46445]
[80.31345]
[71.343]
[69.9066]
[80.4498]
[74.4414]
[85.90215]
[79.60575]
[81.3153]
[78.0987]
[78.0141]
[73.9929]
[79.06485]
[79.9785]
[81.04815]
[73.9743]
[70.0971]
[78.31905]
[74.30625]
[77.49675]
[74.82855]
[70.55895]
[72.3054]
[71.3148]
[73.42725]
[74.9751]
[75.42495]
[72.65025]
[78.8181]
[77.81535]
[70.6911]
[73.63515]
[75.70455]
[75.4239]
[76.689]
[72.31545]
[72.49155]
[74.37465]
[72.12105]
[72.0993]
[72.41775]
[73.94085]
[80.24115]
[75.2127]
[78.5892]
[83.4441]
[72.0033]
[79.18275]
[79.20765]
[79.4268]
[75.6669]
[85.52775]
[80.2428]
[79.5681]
[75.2805]
[76.47015]
[80.65905]
[80.1816]
[85.22895]
[81.5106]
[85.83975]
[77.8581]
[81.23025]
[74.23665]
[63.59325]
[77.6

(1963, 462, 19)

In [134]:
# 测试集: 筛选下去outlier的剩余的数据
X_test = df_test.loc[:, np.hstack([cols_unit, cols_feature, col_y])].copy()

wafer_number_test = len(X_test.groupby(['WAFER_ID', 'Mode']))
max_batch_length = 462
variable_number = len(cols_feature)
print('训练集样本个数: ', wafer_number_test)
print('批次最长长度: ', max_batch_length)
print('变量个数: ', variable_number)

# 将训练集reshape
# reshape test data
# X_test_r第1维是批次个数，第2维是每个批次的采样时间，不足最长采样时间的用0补齐，第3维是字段的个数
X_test_r = np.zeros((wafer_number_test, max_batch_length, variable_number))

y_test = np.zeros((wafer_number_test, 1))
# y_test[0]

wafer_id_mode_test = []

i = 0
for name, group in X_test.groupby(['WAFER_ID', 'Mode']):
    wafer_id_mode_test.append(name)
    one_wafer = group.drop(columns=['WAFER_ID', 'AVG_REMOVAL_RATE', 'Mode'])
    y_test[i] = group['AVG_REMOVAL_RATE'].iloc[0]
    print(y_test[i])
    one_wafer_length = one_wafer.shape[0]
#     if one_wafer_length < max_batch_length:
    df_tmp = pd.DataFrame(np.zeros((max_batch_length-one_wafer_length, len(cols_feature))), columns=cols_feature)
    X_test_r[i, :, :] = pd.concat([one_wafer, df_tmp])        
    i+=1

X_test_r.shape

训练集样本个数:  422
批次最长长度:  462
变量个数:  19
[60.44715]
[57.2523]
[66.9813]
[56.1786]
[60.8757]
[78.17175]
[80.88135]
[80.7204]
[97.10985]
[69.84885]
[82.56405]
[85.19805]
[70.56045]
[70.04175]
[82.56705]
[79.5159]
[81.22455]
[82.10085]
[85.95015]
[96.4422]
[80.00535]
[96.4671]
[79.7391]
[75.29955]
[62.7504]
[78.48405]
[70.2735]
[69.9954]
[71.6763]
[61.11615]
[70.51575]
[65.64585]
[70.7943]
[69.63975]
[73.66365]
[66.5856]
[70.79865]
[85.05525]
[71.688]
[73.9146]
[76.35855]
[74.8848]
[59.187]
[76.0851]
[72.97155]
[90.48555]
[75.1965]
[77.2197]
[69.4038]
[70.5174]
[73.0947]
[68.36535]
[74.7423]
[68.21805]
[72.5712]
[66.47775]
[68.01585]
[63.6735]
[60.4206]
[66.3294]
[67.9998]
[81.3882]
[72.10245]
[72.18225]
[70.51065]
[71.8215]
[82.80315]
[78.0837]
[86.9994]
[84.7029]
[72.47895]
[70.7502]
[71.17155]
[74.5998]
[83.97105]
[74.5941]
[82.5702]
[78.12615]
[78.03345]
[82.17195]
[151.26585]
[154.6929]
[146.26905]
[76.62405]
[78.6405]
[82.12725]
[66.98055]
[63.4305]
[64.84635]
[58.3209]
[80.7609]
[64.38

(422, 462, 19)

<font color='red'>慎重保存操作</font>

In [141]:
# np.save("./data phm 2016/X_train_r_three_mode.npy", X_train_r)
# np.save("./data phm 2016/X_test_r_three_mode.npy", X_test_r)

# with open("./data phm 2016/wafer_id_mode_train_three_mode.pkl", 'wb') as f:
#     pickle.dump(wafer_id_mode_train, f)
    
# with open("./data phm 2016/wafer_id_mode_test_three_mode.pkl", 'wb') as f:
#     pickle.dump(wafer_id_mode_test, f)

# with open("./data phm 2016/cols_feature_three_mode.pkl", 'wb') as f:
#     pickle.dump(cols_feature, f)

# np.save("./data phm 2016/y_train_three_mode.npy", y_train)
# np.save("./data phm 2016/y_test_three_mode.npy", y_test)